# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [22]:
task = '''
        Write a concise but engaging blogpost about
       Current geo-political mess created by british colonial policies. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [23]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [24]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [25]:
print(reply)

Title: Unraveling the Geo-Political Fallout from British Colonial Policies

Centuries later, the remnants of British colonial policies continue to cast a long shadow over the global landscape, contributing to the current geo-political mess. Drawing arbitrary borders, sowing seeds of conflict among ethnic and religious groups, and exploiting resources for their gain have left a trail of instability and tension. From the Middle East to Africa, the repercussions are evident. Understanding this historical context is crucial in navigating today's complex challenges. By acknowledging this legacy, we can work towards peaceful resolutions and a more harmonious world. History must serve as a guide for forging a better future.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [26]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [27]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Current geo-political mess created by british colonial policies. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unraveling the Geo-Political Fallout from British Colonial Policies

Centuries later, the remnants of British colonial policies continue to cast a long shadow over the global landscape, contributing to the current geo-political mess. Drawing arbitrary borders, sowing seeds of conflict among ethnic and religious groups, and exploiting resources for their gain have left a trail of instability and tension. From the Middle East to Africa, the repercussions are evident. Understanding this historical context is crucial in navigating today's complex challenges. By acknowledging this legacy, we can work towards peaceful resolutions and a more harmonious world. History must serve as a 

## Nested chat

In [28]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [29]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [30]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [31]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [32]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [33]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [34]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Current geo-political mess created by british colonial policies. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unraveling the Geo-Political Fallout from British Colonial Policies

Centuries later, the remnants of British colonial policies continue to cast a long shadow over the global landscape, contributing to the current geo-political mess. Drawing arbitrary borders, sowing seeds of conflict among ethnic and religious groups, and exploiting resources for their gain have left a trail of instability and tension. From the Middle East to Africa, the repercussions are evident. Understanding this historical context is crucial in navigating today's complex challenges. By acknowledging this legacy, we can work towards peaceful resolutions and a more harmonious world. History must serve as a 


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated feedback from reviewers:
- Include relevant keywords such as "British colonial legacy," "global geopolitics," and "historical context" to improve search engine visibility.
- Enhance meta title and description with targeted keywords.
- Add subheadings and bullet points for easier readability and keyword optimization.
- Ensure the content accurately portrays the impact of British colonial policies on geo-politics without glorifying or demonizing any specific groups or nations.
- Avoid language that could be interpreted as promoting a specific political agenda or viewpoint regarding current geopolitical issues stemming from colonial history.
- Acknowledge the complexity of historical legacies and emphasize peaceful resolutions and understanding without oversimplifying or downplaying ongoing challenges in affected regions.

Final suggestion:
The writing should focus on i

## Get the summary

In [35]:
print(res.summary)

Title: Navigating the Impact of British Colonial Legacy on Global Geopolitics

Meta Title: Understanding British Colonial Legacy: Impact on Global Geopolitics
Meta Description: Explore the historical context of British colonial policies and their enduring impact on today's global geopolitics. Learn about the complexities and strive towards peaceful resolutions.

Subheading: Unraveling Historical Context
- British Colonial Legacy: Arbitrary Borders and Ethnic Divisions
- Resource Exploitation and Economic Imbalances

Subheading: Impact on Global Geopolitics
- Middle East: Sectarian Tensions and Political Instability
- Africa: Struggles with Post-Colonial Governance

In today's global geopolitics, the shadows of British colonial policies loom large, shaping narratives and fueling tensions. By acknowledging this complex historical legacy with nuance and balance, we can strive for peaceful resolutions and a better understanding of the challenges faced in affected regions. Let history guide